In [61]:
def category_counts(data):
    """
    Count total number of unique DepartmentDescription made on each trip.
    """
    counts = []
    for array in np.asarray(data.loc[:, "1-HR PHOTO":"WIRELESS"]):
        count = 0
        for item in array:
            if item > 0:
                count += 1
        counts.append(count)
    cat_counts = pd.DataFrame(counts)
    cat_counts = cat_counts.rename(columns={0:"CategoryCount"})
    cat_counts = cat_counts.set_index(data.index)
    
    data.insert(6, "CategoryCounts", cat_counts)
    
    return data

In [62]:
def company(x):
    try:
        p = x[:6]
        if p == "000000":
            return x[-5]
        return p
    except:
        return -9999
    
def float_to_str(obj):
    while obj != "nan":
        obj = str(obj).split(".")[0]
        return obj

In [70]:
from collections import Counter
def mode(x):
    counts = Counter(x)
    max_count = max(counts.values())
    ls = [x_i for x_i, count in counts.items() if count == max_count]
    return ls[0]

In [63]:
train = pd.read_csv('../dataset/train.csv')
test = pd.read_csv('../dataset/test.csv')

In [64]:
train["Upc"] = train.Upc.apply(float_to_str)
test["Upc"] = test.Upc.apply(float_to_str)

train["company"] = train.Upc.apply(company) 
test["company"] = test.Upc.apply(company)

In [71]:
train.loc[train["ScanCount"] < 0, "Return"] = 1
train.loc[train["Return"] != 1, "Return"] = 0

test.loc[test["ScanCount"] < 0, "Return"] = 1
test.loc[test["Return"] != 1, "Return"] = 0

In [72]:
wd = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, 
      "Friday": 4, "Saturday": 5, "Sunday": 6}

train["Weekday"] = train["Weekday"].apply(lambda x: wd[x])
test["Weekday"] = test["Weekday"].apply(lambda x: wd[x])

In [73]:
train["Pos_Sum"] = train["ScanCount"]
test["Pos_Sum"] = test["ScanCount"]

train.loc[train["Pos_Sum"] < 0, "Pos_Sum"] = 0
test.loc[test["Pos_Sum"] < 0, "Pos_Sum"] = 0

In [74]:
train["Neg_Sum"] = train["ScanCount"]
test["Neg_Sum"] = test["ScanCount"]

train.loc[train["Neg_Sum"] > 0, "Neg_Sum"] = 0
test.loc[test["Neg_Sum"] > 0, "Neg_Sum"] = 0

In [75]:
train = train.drop(["Upc", "FinelineNumber"], axis=1)
test = test.drop(["Upc", "FinelineNumber"], axis=1)

In [76]:
train_dd = pd.get_dummies(train["DepartmentDescription"])
test_dd = pd.get_dummies(test["DepartmentDescription"])

train_dd = pd.concat([train[["VisitNumber"]], train_dd], axis=1)
test_dd = pd.concat([test[["VisitNumber"]], test_dd], axis=1)

train_dd = train_dd.groupby("VisitNumber", as_index=False).sum()
test_dd = test_dd.groupby("VisitNumber", as_index=False).sum()

In [77]:
train_company = train[["VisitNumber", "company"]]
test_company = test[["VisitNumber", "company"]]

In [78]:
train_company = train_company.groupby("VisitNumber", as_index=False).agg(mode)
test_company = test_company.groupby("VisitNumber", as_index=False).agg(mode)

In [79]:
train_dd.shape, test_dd.shape

((95674, 69), (95674, 68))

In [80]:
train_company.shape, test_company.shape

((95674, 2), (95674, 2))

In [81]:
train_by_sum = train[["VisitNumber", "ScanCount", "Pos_Sum", "Neg_Sum"]]
test_by_sum = test[["VisitNumber", "ScanCount", "Pos_Sum", "Neg_Sum"]]

train_by_max = train[["TripType", "VisitNumber", "Weekday", "Return"]]
test_by_max = test[["VisitNumber", "Weekday", "Return"]]

In [82]:
train_by_sum = train_by_sum.groupby("VisitNumber", as_index=False).sum()
test_by_sum = test_by_sum.groupby("VisitNumber", as_index=False).sum()

train_by_max = train_by_max.groupby("VisitNumber", as_index=False).max()
test_by_max = test_by_max.groupby("VisitNumber", as_index=False).max()

In [83]:
train = train_by_sum.merge(train_by_max, on=["VisitNumber"])
train = train.merge(train_dd, on=["VisitNumber"])
train = train.merge(train_company, on=["VisitNumber"])

test = test_by_sum.merge(test_by_max, on=["VisitNumber"])
test = test.merge(test_dd, on=["VisitNumber"])
test = test.merge(test_company, on=["VisitNumber"])

In [84]:
y = train["TripType"]
train = train.drop(["TripType", "HEALTH AND BEAUTY AIDS"], axis=1)

In [85]:
train = category_counts(train)
test = category_counts(test)

In [86]:
# train["R_UniqueDepts"] = train["CategoryCounts"] / train["Pos_Sum"]
# test["R_UniqueDepts"] = test["CategoryCounts"] / test["Pos_Sum"]

In [87]:
train = pd.get_dummies(train, columns=["Weekday", "Return"])
test = pd.get_dummies(test, columns=["Weekday", "Return"])

In [88]:
train.shape, test.shape

((95674, 82), (95674, 82))

In [89]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

In [90]:
label_enc = LabelEncoder().fit(y)
y_labeled = label_enc.transform(y)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(
    train, y_labeled, random_state=0)

In [92]:
dtrain = xgb.DMatrix(X_train.values, label=y_train)
dtest = xgb.DMatrix(X_test.values, label=y_test)

In [93]:
num_boost_round = 300
params = {'objective': 'multi:softprob', 
          'eval_metric': 'mlogloss',
          'num_class':38, 
          'max_delta_step': 3, 
          'eta': 0.25}

evals = [(dtrain, 'train'), (dtest, 'eval')]


bst = xgb.train(params=params,  
                dtrain=dtrain, 
                num_boost_round=num_boost_round, 
                evals=evals,
               early_stopping_rounds=10,)

[0]	train-mlogloss:2.93853	eval-mlogloss:2.94907
Multiple eval metrics have been passed: 'eval-mlogloss' will be used for early stopping.

Will train until eval-mlogloss hasn't improved in 10 rounds.
[1]	train-mlogloss:2.3537	eval-mlogloss:2.37968
[2]	train-mlogloss:1.93129	eval-mlogloss:1.97088
[3]	train-mlogloss:1.68169	eval-mlogloss:1.73097
[4]	train-mlogloss:1.50958	eval-mlogloss:1.56709
[5]	train-mlogloss:1.37949	eval-mlogloss:1.44493
[6]	train-mlogloss:1.2791	eval-mlogloss:1.35178
[7]	train-mlogloss:1.19941	eval-mlogloss:1.27848
[8]	train-mlogloss:1.13369	eval-mlogloss:1.21884
[9]	train-mlogloss:1.07919	eval-mlogloss:1.17012
[10]	train-mlogloss:1.0328	eval-mlogloss:1.1296
[11]	train-mlogloss:0.993882	eval-mlogloss:1.09621
[12]	train-mlogloss:0.960401	eval-mlogloss:1.06792
[13]	train-mlogloss:0.931296	eval-mlogloss:1.04425
[14]	train-mlogloss:0.905878	eval-mlogloss:1.02368
[15]	train-mlogloss:0.88251	eval-mlogloss:1.0053
[16]	train-mlogloss:0.862223	eval-mlogloss:0.989796
[17]	tra

In [94]:
classes = np.array(list(set(label_enc.inverse_transform(y_labeled))))

In [95]:
dmtest = xgb.DMatrix(test.values) 
pred_proba = bst.predict(dmtest)

proba_df = pd.DataFrame(pred_proba, columns=classes)
proba_df.columns = proba_df.columns.map(lambda x: "TripType_" + str(x))
sub_df = pd.concat([test[["VisitNumber"]], proba_df], axis=1)
sub_df.to_csv("../submission_0410_noProb_xgb_01.csv", index=False)